# Instalación Desatendida de SQL Server

## Puesta a punto

Este archivo esta configurado para hacer la instalación de un SQL Server Stand alone

### Directorio

Se crean las carpetas a usar por el SQL Server (teniendo en cuenta ls recursos disponibles en el servidor), el recurso compartido para la extracción del bak, se asigna permiso al usuario do\bkpsuser que levanta los bak para mantener la estructura aph.

### Puerto 1433

Se habilita el puerto para permitir conexiones al SQL Server

In [ ]:
[string]$usuario = Read-Host -Prompt "Ingrese el nombre de usuario"
$passdo = Read-Host -Prompt "Ingrese la contraseña del usuario" -AsSecureString
$credo = New-Object -TypeName System.Management.Automation.PSCredential ($usuario, $passdo)

Invoke-Command -ComputerName "SQLTESTDBA" -Credential $credo -ScriptBlock{ `
New-Item -Path 'D:\BDs' -ItemType "directory" 
New-Item -Path 'E:\Backups' -ItemType "directory"
New-Item -Path 'E:\Backups\Diario' -ItemType "directory"
New-Item -Path 'E:\Tempdb' -ItemType "directory"

New-SmbShare -Name  "Diario" -Path "E:\Backups\Diario" -FullAccess "dominio\bkpsuser"

New-NetFirewallRule -DisplayName "AllowSQLConn" -Direction Inbound -LocalPort 1433 -Protocol TCP -Action Allow} 

# Instalación de SQL Server desde CMD

Tener en cuenta para el archivo configuration.ini lo siguiente:

- Cuenta de Servicio del motor: dominio\cuentaServicioSQL
    
- Cuenta de servicio del agente: dominio\cuentaAgenteSQL
    
- Autenticación Mixta.
    
- Agregar usuario sysadmin: dominio\cuentasysadmin (no current user)
    
- Mapear las carpetas creadas en pasos previos.
    
- Collation: SQL_Latin1_General_CP850_CI_AI
    
- MAXDOP, Memoria y Temp cantidades sugeridas por el instalador (se pueden modificar luego)
    
- TempDB: indicar carpeta de archivos

Con el archivo ConfigurationFile.ini configurado según las necesidades, ejecutar en un cmd elevado, ubicado en donde se encuentre montada la imagen de SQL Server:

Setup.exe /PID="Licencia" /IACCEPTSQLSERVERLICENSETERMS /SAPWD="**********" /SQLSYSADMINACCOUNTS="do\sqlsadm" /SQLSVCPASSWORD="***********" /AGTSVCPASSWORD="**************" /ConfigurationFile="C:\Conf\ConfigurationFileTest.ini"

## ConfigurationFile ejemplo: 

;SQL Server 2019 Configuration File

[OPTIONS]

; By specifying this parameter and accepting Microsoft R Open and Microsoft R Server terms, you acknowledge that you have read and understood the terms of use.

IACCEPTPYTHONLICENSETERMS="False"

; Specifies a Setup work flow, like INSTALL, UNINSTALL, or UPGRADE. This is a required parameter.

ACTION="Install"

; By specifying this parameter and accepting Microsoft R Open and Microsoft R Server terms, you acknowledge that you have read and understood the terms of use.

IACCEPTROPENLICENSETERMS="False"

; Specifies that SQL Server Setup should not display the privacy statement when ran from the command line.

SUPPRESSPRIVACYSTATEMENTNOTICE="False"

; Use the /ENU parameter to install the English version of SQL Server on your localized Windows operating system.

ENU="True"

; Setup will not display any user interface.

QUIET="False"

; Setup will display progress only, without any user interaction.

QUIETSIMPLE="True"

; Parameter that controls the user interface behavior. Valid values are Normal for the full UI,AutoAdvance for a simplied UI, and EnableUIOnServerCore for bypassing Server Core setup GUI block.

;UIMODE="Normal"

; Specify whether SQL Server Setup should discover and include product updates. The valid values are True and False or 1 and 0. By default SQL Server Setup will include updates that are found.

UpdateEnabled="True"

; If this parameter is provided, then this computer will use Microsoft Update to check for updates.

USEMICROSOFTUPDATE="False"

; Specifies that SQL Server Setup should not display the paid edition notice when ran from the command line.

SUPPRESSPAIDEDITIONNOTICE="False"

; Specify the location where SQL Server Setup will obtain product updates. The valid values are "MU" to search Microsoft Update, a valid folder path, a relative path such as .\MyUpdates or a UNC share. By default SQL Server Setup will search Microsoft Update or a Windows Update service through the Window Server Update Services.

UpdateSource="MU"

; Specifies features to install, uninstall, or upgrade. The list of top-level features include SQL, AS, IS, MDS, and Tools. The SQL feature will install the Database Engine, Replication, Full-Text, and Data Quality Services (DQS) server. The Tools feature will install shared components.

FEATURES=SQLENGINE,FULLTEXT,CONN,IS,BC,SDK,SNAC_SDK

; Displays the command line parameters usage

HELP="False"

; Specifies that the detailed Setup log should be piped to the console.

INDICATEPROGRESS="False"

; Specifies that Setup should install into WOW64. This command line argument is not supported on an IA64 or a 32-bit system.

X86="False"

; Specify a default or named instance. MSSQLSERVER is the default instance for non-Express editions and SQLExpress for Express editions. This parameter is required when installing the SQL Server Database Engine (SQL), or Analysis Services (AS).

INSTANCENAME="MSSQLSERVER"

; Specify the root installation directory for shared components. This directory remains unchanged after shared components are already installed.

INSTALLSHAREDDIR="C:\Program Files\Microsoft SQL Server"

; Specify the root installation directory for the WOW64 shared components. This directory remains unchanged after WOW64 shared components are already installed.

INSTALLSHAREDWOWDIR="C:\Program Files (x86)\Microsoft SQL Server"

; Specify the Instance ID for the SQL Server features you have specified. SQL Server directory structure, registry structure, and service names will incorporate the instance ID of the SQL Server instance.

INSTANCEID="MSSQLSERVER"

; TelemetryUserNameConfigDescription

;SQLTELSVCACCT="NT Service\SQLTELEMETRY$SQL2019"

; TelemetryStartupConfigDescription

SQLTELSVCSTARTUPTYPE="Disabled"

; TelemetryStartupConfigDescription

;ISTELSVCSTARTUPTYPE="Disabled"

; TelemetryUserNameConfigDescription

;ISTELSVCACCT="NT Service\SSISTELEMETRY150"

; Specify the installation directory.

INSTANCEDIR="C:\Program Files\Microsoft SQL Server"

; Agent account name

AGTSVCACCOUNT=<strong>"Dominio\cuentaAgenteSQL"</strong>

; Auto-start service after installation.

AGTSVCSTARTUPTYPE=<strong>"Automatic"</strong>

; Startup type for Integration Services.

ISSVCSTARTUPTYPE=<strong>"Automatic"</strong>

; Account for Integration Services: Domain\User or system account.

ISSVCACCOUNT="NT Service\MsDtsServer150"

; CM brick TCP communication port

COMMFABRICPORT="0"

; How matrix will use private networks

COMMFABRICNETWORKLEVEL="0"

; How inter brick communication will be protected

COMMFABRICENCRYPTION="0"

; TCP port used by the CM brick

MATRIXCMBRICKCOMMPORT="0"

; Startup type for the SQL Server service.

SQLSVCSTARTUPTYPE="Automatic"

; Level to enable FILESTREAM feature at (0, 1, 2 or 3).

FILESTREAMLEVEL="0"

; The max degree of parallelism (MAXDOP) server configuration option.

SQLMAXDOP="0"

; Set to "1" to enable RANU for SQL Server Express.

ENABLERANU="False"

; Specifies a Windows collation or an SQL collation to use for the Database Engine.

SQLCOLLATION="SQL_Latin1_General_CP850_CI_AI"

; Account for SQL Server service: Domain\User or system account.

SQLSVCACCOUNT=<strong>"Dominio\cuentaServicioSQL"</strong>

; Set to "True" to enable instant file initialization for SQL Server service. If enabled, Setup will grant Perform Volume Maintenance Task privilege to the Database Engine Service SID. This may lead to information disclosure as it could allow deleted content to be accessed by an unauthorized principal.

SQLSVCINSTANTFILEINIT="True"

; Windows account(s) to provision as SQL Server system administrators.

SQLSYSADMINACCOUNTS=<strong>"Dominio\cuentasyssadm"</strong>

; The default is Windows Authentication. Use "SQL" for Mixed Mode Authentication.

SECURITYMODE=<strong>"SQL"</strong>

; The number of Database Engine TempDB files.

SQLTEMPDBFILECOUNT="8"

; Specifies the initial size of a Database Engine TempDB data file in MB.

SQLTEMPDBFILESIZE="8"

; Specifies the automatic growth increment of each Database Engine TempDB data file in MB.

SQLTEMPDBFILEGROWTH="64"

; Specifies the initial size of the Database Engine TempDB log file in MB.

SQLTEMPDBLOGFILESIZE="8"

; Specifies the automatic growth increment of the Database Engine TempDB log file in MB.

SQLTEMPDBLOGFILEGROWTH="64"

; Default directory for the Database Engine backup files.

SQLBACKUPDIR=<strong>"E:\Backups"</strong>

; Default directory for the Database Engine user databases.

SQLUSERDBDIR=<strong>"D:\BDs"</strong>

; Directories for Database Engine TempDB files.

SQLTEMPDBDIR=<strong>"E:\Tempdb"</strong>

; Provision current user as a Database Engine system administrator for %SQL_PRODUCT_SHORT_NAME% Express.

ADDCURRENTUSERASSQLADMIN="False"

; Specify 0 to disable or 1 to enable the TCP/IP protocol.

TCPENABLED="1"

; Specify 0 to disable or 1 to enable the Named Pipes protocol.

NPENABLED="0"

; Startup type for Browser Service.

BROWSERSVCSTARTUPTYPE="Disabled"

; Add description of input argument FTSVCACCOUNT

FTSVCACCOUNT="NT Service\MSSQLFDLauncher"

; Use USESQLRECOMMENDEDMEMORYLIMITS to minimize the risk of the OS experiencing detrimental memory pressure.

USESQLRECOMMENDEDMEMORYLIMITS="True"

# Instalación de SQL Server desde Powershell

## Requerimientos

Descargar e instalar las funciones: 

-   Get-IniContent: [https://gallery.technet.microsoft.com/ea40c1ef-c856-434b-b8fb-ebd7a76e8d91](https://gallery.technet.microsoft.com/ea40c1ef-c856-434b-b8fb-ebd7a76e8d91)
-   Out-IniFile: [https://gallery.technet.microsoft.com/7d7c867f-026e-4620-bf32-eca99b4e42f4](https://gallery.technet.microsoft.com/7d7c867f-026e-4620-bf32-eca99b4e42f4)

## Preparación

Teniendo el ConfigurationFile.ini en el pc particular, no en el servidor:

Configurar las rutas y el usuario para levantar los archivos bak

In [ ]:
$IniFile = Get-IniContent -FilePath "C:\InisSQL\ConfigurationFile.ini"

$IniFile.OPTIONS.SQLUSERDBDIR    = '"D:\BDs"'

$IniFile.OPTIONS.SQLBACKUPDIR    = '"E:\Backups"'

$IniFile.OPTIONS.SQLUSERDBLOGDIR = '"D:\BDs"'

$IniFile.OPTIONS.SQLTEMPDBDIR    = '"E:\TempDB"'

$IniFile | Out-IniFile -FilePath "C:\InisSQL\Resultado.ini" -Force

[string]$usuario = Read-Host -Prompt "Ingrese el nombre de usuario"
$passdo = Read-Host -Prompt "Ingrese la contraseña del usuario" -AsSecureString
$credo = New-Object -TypeName System.Management.Automation.PSCredential ($usuario, $passdo)

Invoke-Command -ComputerName "SQLTESTDBA" -Credential $credo -ScriptBlock{ `

                                New-Item -Path 'D:\BDs' -ItemType "directory" 

                                New-Item -Path 'E:\Backups' -ItemType "directory"

                                New-Item -Path 'E:\Backups\Diario' -ItemType "directory"

                                New-Item -Path 'E:\Tempdb' -ItemType "directory"

                               New-SmbShare -Name "Diario" -Path "E:\Backups\Diario" -FullAccess "dominio\bkpsuser" }

New-PSDrive -Name SQLremoto -PSProvider FileSystem -Root "\\SQLTESTDBA\e$" -Credential $credo
Copy-Item -Path "C:\InisSQL\Resultado.ini" -Destination "SQLremoto:"

## Instalación

En el servidor

In [ ]:
$ArgumentList =  @()
$ArgumentList += '/PID="Licencia SQL"'
$ArgumentList += '/SAPWD="Escribir la contraseña"'
$ArgumentList += '/SQLSYSADMINACCOUNTS="dominio\cuentasysadm"'
$ArgumentList += '/SQLSVCPASSWORD="**********"'
$ArgumentList += '/AGTSVCPASSWORD="***********"'
$ArgumentList += '/ConfigurationFile="E:\Resultado.ini"'
$ArgumentList += '/IAcceptSqlServerLicenseTerms'
Start-Process  -FilePath "d:\setup.exe" -ArgumentList $ArgumentList -Wait

#  Configurar SPN

Teniendo el módulo dbatools se configura el SPN.

In [ ]:
Set-DbaSpn -SPN MSSQLSvc/SQLSERVER.do.scba.gov.ar -ServiceAccount dominio\cuentaservicioSQL -Credential $credo

# Habilitar Compresión de  Backups

Teniendo el módulo dbatools, primero crear la credencial para loguearse a la instancia, luego habilitar la compresión.

Finalmente, se comprueba el cambio.

In [ ]:
[string]$usuario = Read-Host -Prompt "Ingrese el nombre de usuario"
$passdo = Read-Host -Prompt "Ingrese la contraseña del usuario" -AsSecureString
$credo = New-Object -TypeName System.Management.Automation.PSCredential ($usuario, $passdo)


Set-DbaSpConfigure -SqlInstance VMTestDBA -SqlCredential $credential -Name 'backup compression default' -Value 1
Get-DbaSpConfigure -SqlInstance vmtestdba -SqlCredential $credential -Name 'backup compression default' | Select-Object displayName, RunningValue |Format-Table

# Crear un nuevo Login SQL SYSADMIN y deshabilitar SA

In [ ]:
$usersql = Read-Host "Ingrese nombre del usuario SQL"
$securePassword = Read-Host "Ingrese la contraseñe" -AsSecureString
New-DbaLogin -SqlInstance $SqlInstance -SqlCredential $credential -Login $usersql -SecurePassword $securePassword -Force
Invoke-DbaQuery  -SqlInstance $SqlInstance -SqlCredential $credential -File "C:\InstalacionSqlServer\HandlingSA.sql" 
$credusersql = New-Object -TypeName System.Management.Automation.PSCredential ($usuario, $securePassword)

# Preparar los SP para el Plan de Mantenimiento

También usando dbatools. Luego está la comprobación.

In [ ]:
Invoke-DbaQuery  -SqlInstance vmtestdba -SqlCredential $credusersql -File "C:\InstalacionSQLServer\PMSolucionParametrizada.sql"
Invoke-DbaQuery  -SqlInstance vmtestdba -SqlCredential $credusersql -Query "Select name from sys.procedures"

# Configurar DataBaseMail y Operador

In [ ]:
Invoke-DbaQuery  -SqlInstance $SqlInstance -SqlCredential $credusersql -File "C:\InstalacionSQLServer\DatabaseMailYOperadorConfiguracion.sql"